# Imports

In [65]:
import os
import glob
import shutil
from collections import Counter

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import numpy as np
from tqdm import tqdm

import sklearn.metrics

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision import transforms
import torchvision

In [89]:
CORE_PATH = "/content/drive/MyDrive/data"
ORIG_PATH = "/content/drive/MyDrive/data/emotions_orig"
TO_LOAD_PATH = "/content/drive/MyDrive/data/emotions_load"

# EDA

In [3]:
emotions = os.listdir(ORIG_PATH)
emotions

['fear', 'sadness', 'disgust', 'contempt', 'anger', 'joy', 'wonder']

In [ ]:
primary_distribution = {x: 0 for x in emotions}

for emote in emotions:
    files = [f for f in os.listdir(f"{ORIG_PATH}/{emote}/")]
    primary_distribution[emote] = len(files)

fig = go.Figure(
    go.Bar(x=list(primary_distribution.keys()),
           y=list(primary_distribution.values())),
)
fig.update_layout(
    width=600,
    height=400,
    title="Number of images"
)
fig.show(renderer="colab")

In [ ]:
sum(primary_distribution.values())

496

# Data Splitting

In [82]:
np.random.seed(2024)
test_size = .1

train_samples = {}
test_samples = {}

for emote in emotions:
    for dirpath, _, filenames in os.walk(f"{ORIG_PATH}/{emote}"):
        files = [os.path.abspath(os.path.join(dirpath, f)) for f in filenames]

    np.random.shuffle(files)
    test_num = int(len(files) * test_size)

    train_samples[emote] = files[:-test_num]
    test_samples[emote] = files[-test_num:]

In [83]:
for emote in emotions:
    os.makedirs(f"{TO_LOAD_PATH}/train/{emote}")
    os.makedirs(f"{TO_LOAD_PATH}/test/{emote}")

    for train_file in train_samples[emote]:
        shutil.copy(train_file, f"{TO_LOAD_PATH}/train/{emote}")
    for test_file in test_samples[emote]:
        shutil.copy(test_file, f"{TO_LOAD_PATH}/test/{emote}")

# Data balancing and loading

In [4]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomGrayscale(),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

inf_transforms = transforms.Compose([transforms.Resize(224),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

train = torchvision.datasets.ImageFolder(
    f"{TO_LOAD_PATH}/train",
    transform=train_transforms
)
test = torchvision.datasets.ImageFolder(
    f"{TO_LOAD_PATH}/test",
    transform=inf_transforms
)

In [5]:
len(train) + len(test)

496

In [6]:
train.class_to_idx, test.class_to_idx

({'anger': 0,
  'contempt': 1,
  'disgust': 2,
  'fear': 3,
  'joy': 4,
  'sadness': 5,
  'wonder': 6},
 {'anger': 0,
  'contempt': 1,
  'disgust': 2,
  'fear': 3,
  'joy': 4,
  'sadness': 5,
  'wonder': 6})

In [7]:
def get_loader(image_folder, BATCH_SIZE):
    distrib = Counter(image_folder.targets)
    class_weights = {x: len(image_folder) / distrib[x] for x in distrib.keys()}
    sample_weights = [class_weights[x] for x in image_folder.targets]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

    return DataLoader(image_folder, BATCH_SIZE, sampler=sampler)

In [8]:
train_loader = get_loader(train, 16)
test_loader = get_loader(test, 16)

# Utility functions

In [9]:
def train_model(model, train_loader, val_loader, epochs, criterion, optimizer, device):
    model.to(device)
    train_loss = []
    val_loss = []
    train_batch_accuracy = []
    val_batch_accuracy = []

    for step, epoch in enumerate(range(epochs)):
        loss_per_batch = 0.0

        model.train()
        for image, label in tqdm(train_loader):
            optimizer.zero_grad()
            image, label = image.to(device), label.to(device)

            output = model(image)
            output_pred = output.max(1)[1].data
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            loss_per_batch += loss.item() * output.shape[0]
            train_batch_accuracy.append(np.mean((label.cpu() == output_pred.cpu()).numpy() ))

        train_loss.append(loss_per_batch / len(train_loader.dataset))

        print(f"EPOCH: {step + 1}, train_loss: {train_loss[step]}, train_acc: {np.mean(train_batch_accuracy)}")
        print()

In [10]:
def test_model(model, test_loader, device):
    model.eval()
    test_batch_acc = []
    for X_batch, y_batch in tqdm(test_loader):
        logits = model(X_batch.to(device))
        y_pred = logits.max(1)[1].data
        test_batch_acc.append(np.mean((y_batch.cpu() == y_pred.cpu()).numpy()))

    test_accuracy = np.mean(test_batch_acc)
    print(f"Test accuracy:\t\t{test_accuracy * 100:.2f} %")

In [61]:
def inference_model(model, folder, device):
    data = torchvision.datasets.ImageFolder(
        folder,
        transform=inf_transforms
    )
    model.to(device)
    model.eval()

    true = []
    pred = []
    for image, label in data:
        logits = model(image.to(device).unsqueeze(0))

        pred.append(logits.max(1)[1].data.cpu().item())
        true.append(label)

    return true, pred

In [75]:
# source: https://www.kaggle.com/code/nkitgupta/evaluation-metrics-for-multi-class-classification
def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):
    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        other_class = [x for x in unique_class if x != per_class]
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]
        roc_auc = sklearn.metrics.roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc

    return roc_auc_dict

def get_metrics_report(true, pred):
    return {
        "Accuracy": sklearn.metrics.accuracy_score(true, pred),
        "Precision_macro": sklearn.metrics.precision_score(true, pred, average='macro'),
        "Precision_micro": sklearn.metrics.precision_score(true, pred, average='micro'),
        "Recall_macro": sklearn.metrics.recall_score(true, pred, average='macro'),
        "Recall_micro": sklearn.metrics.recall_score(true, pred, average='micro'),
        "ROC_AUC": roc_auc_score_multiclass(true, pred)
    }

# RESNET

In [11]:
from torchvision.models import resnet50, ResNet50_Weights

In [12]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

for name, param in model.named_parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(model.fc.in_features, 512),
                            nn.ReLU(),
                            nn.Dropout(),
                            nn.Linear(512, len(emotions)))
model.fc = classifier

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


In [16]:
EPOCHS = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_model(model, train_loader, test_loader, EPOCHS, criterion, optimizer, device)

100%|██████████| 29/29 [00:12<00:00,  2.24it/s]


EPOCH: 1, train_loss: 1.918154629938852, train_acc: 0.22198275862068967



100%|██████████| 29/29 [00:12<00:00,  2.40it/s]


EPOCH: 2, train_loss: 1.8311815458310474, train_acc: 0.24676724137931033



100%|██████████| 29/29 [00:12<00:00,  2.33it/s]


...

EPOCH: 98, train_loss: 0.7796373834588746, train_acc: 0.5827322308233638



100%|██████████| 29/29 [00:12<00:00,  2.23it/s]


EPOCH: 99, train_loss: 0.8664766165089767, train_acc: 0.5834639498432602



100%|██████████| 29/29 [00:12<00:00,  2.26it/s]

EPOCH: 100, train_loss: 0.7794344634946045, train_acc: 0.584698275862069



In [ ]:
test_model(model, test_loader, device)

100%|██████████| 3/3 [00:01<00:00,  2.39it/s]

Test accuracy:		42.64 %


In [ ]:
torch.save(model, "resnet50_100.pt")

In [17]:
model_path = "/content/drive/MyDrive/data/resnet50_100.pt"
model = torch.load(model_path)

In [78]:
true, pred = inference_model(model, f"{TO_LOAD_PATH}/test", device)

In [88]:
get_metrics_report(true, pred)

{'Accuracy': 0.3829787234042553,
 'Precision_macro': 0.41800659657802514,
 'Precision_micro': 0.3829787234042553,
 'Recall_macro': 0.32987528344671196,
 'Recall_micro': 0.3829787234042553,
 'ROC_AUC': {0: 0.6,
  1: 0.5934959349593496,
  2: 0.46590909090909094,
  3: 0.45993589743589747,
  4: 0.8362573099415205,
  5: 0.6695906432748538,
  6: 0.6642857142857144}}